In [1]:
import json

In [2]:
import gzip

In [3]:
from tqdm.notebook import tqdm

In [4]:
def normalize_text(t):
    temp = str.lower(t)
    tokens = [".", ",", ":", "(", ")", ";", "!", "[", "]", "/"]
    for token in tokens:
        temp = temp.replace(token, "")
    return temp

In [ ]:
root = "/calcul/datasets/s2orc/20200705v1/full/metadata/"
with open(root + "title_abstract_processed.txt", "w") as fout:    
    with open(root + "metadata_processed.txt", "w") as fout2:
        for archive in os.listdir(root):
            if archive.endswith(".gz"):
                with gzip.open(root + archive, 'rb') as fin:
                    for line in fin:
                        paper = json.loads(line.decode('utf-8'))
                        if paper["abstract"] is not None:
                            fout.write(normalize_text(paper['title']) + " " + normalize_text(paper['abstract']) + "\n")
                            fout2.write(json.dumps({"paper_id" : paper["paper_id"], 
                                         "authors" : paper["authors"], 
                                         "venue" : paper["venue"], 
                                         "journal" : paper["journal"], 
                                         "mag_id" : paper["mag_id"], 
                                         "mag_field" : paper["mag_field_of_study"]}) + "\n")